In [3]:
# Install dependencies
!pip install diffusers transformers accelerate scipy safetensors

# Import libraries
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch
from PIL import Image
import matplotlib.pyplot as plt

# Define prompt
prompt = "A serene mountain lake at sunrise, ultra realistic, 4K"  # Change your prompt here

# Check device availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load model with Euler sampler
model_id = "runwayml/stable-diffusion-v1-5"
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")

# Adjust dtype based on device
if device == "cuda":
    pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
else:
    # Use float32 for CPU to avoid potential issues
    pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float32)

pipe = pipe.to(device)

# Disable NSFW checker if needed
pipe.safety_checker = lambda images, **kwargs: (images, [False] * len(images))

# Enable memory efficient attention for better performance
try:
    pipe.enable_attention_slicing()
except:
    print("Attention slicing not available")

# Skip CPU offload entirely to avoid CUDA initialization issues
print("Skipping CPU offload to avoid CUDA conflicts")

# Define step values to test (reduced for CPU efficiency)
if device == "cpu":
    steps_to_test = [10, 20, 30]  # Fewer steps for CPU
    print("Running on CPU - using fewer steps for faster execution")
else:
    steps_to_test = [10, 20, 30, 40, 50]

# Generate images
images = []
print("Generating images...")
for i, steps in enumerate(steps_to_test):
    print(f"Generating image {i+1}/{len(steps_to_test)} with {steps} steps...")

    # Set generator for reproducible results
    generator = torch.Generator(device=device).manual_seed(42)

    image = pipe(
        prompt,
        num_inference_steps=steps,
        guidance_scale=7.5,
        generator=generator
    ).images[0]

    images.append((steps, image))

# Plot images
plt.figure(figsize=(20, 10))
for i, (steps, img) in enumerate(images):
    plt.subplot(1, len(images), i + 1)
    plt.imshow(img)
    plt.title(f"{steps} Steps")
    plt.axis("off")
plt.tight_layout()
plt.show()

print("Image generation complete!")

Output hidden; open in https://colab.research.google.com to view.